In [1]:
import requests

# 網址
site = "https://query1.finance.yahoo.com/v8/finance/chart/2330.TW?period1=0&period2=1549258857&interval=1d&events=history&=hP2rOschxO0"

# 利用 requests 來跟遠端 server 索取資料
response = requests.get(site)

print(response.text)

import json
import numpy as np
import pandas as pd

data = json.loads(response.text)
df = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0], index=pd.to_datetime(np.array(data['chart']['result'][0]['timestamp'])*1000*1000*1000))

df.head()
df.close.plot()

Forbidden


JSONDecodeError: ignored

https://www.finlab.tw/%E7%94%A8%E7%88%AC%E8%9F%B2%E7%88%AC%E5%85%A8%E4%B8%96%E7%95%8C%E8%82%A1%E5%83%B9/

用 Python 打造自己的股市資料

  Yahoo Finance：主要的資料來源處 / 美股資料品質不錯
  FRED：滿滿的總體經濟數據
  Google Trends：可獲得股票關鍵字搜尋流行度的資料




In [ ]:
pip install yfinance #Yahoo Finance python API
pip install fredapi #FRED python API
pip install pytrends #Google Trends python API

import pandas as pd

# 貼上連結
url = 'https://finance.yahoo.com/screener/predefined/most_actives?count=100&offset=0'
data = pd.read_html(url)[0]

# 欄位『Symbol』就是股票代碼
stk_list = data.Symbol
stk_list

import pandas as pd

# 貼上連結
url = 'https://www.slickcharts.com/sp500'
data = pd.read_html(url)[0]

# 欄位『Symbol』就是股票代碼
stk_list = data.Symbol

# 用 replace 將符號進行替換
stk_list = data.Symbol.apply(lambda x: x.replace('.', '-'))

import requests
import pandas as pd
# 貼上連結
url = 'https://www.slickcharts.com/sp500'
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

request = requests.get(url, headers = headers)

data = pd.read_html(request.text)[0]

# 欄位『Symbol』就是股票代碼
stk_list = data.Symbol

# 用 replace 將符號進行替換
stk_list = data.Symbol.apply(lambda x: x.replace('.', '-'))
stk_list

# 必要步驟！告訴 python 你的標的股票是什麼
stock = yf.Ticker('股票代碼')
# 取得各種資料
stock.info # 取得公司資料
stock.financials # 取得損益表
stock.balance_sheet # 取得資產負債表
stock.cashflow # 取得現金流量表
stock.history # 取得價量資料＋股利發放資料＋股票分割資料

GET 公司資料

In [2]:
import yfinance as yf
import time

# 取得個股公司資料的語法，先測試一檔看看
stk_basic_data = yf.Ticker('AAPL').info
stk_basic_data

# 將 yfinance 有提供的數據項目取出存在 info_columns，它將會成為 stk_info_df 這張總表的欄位項目
info_columns = list(stk_basic_data.keys())

# 創立一個名為 stk_info_df 的總表，用來存放所有股票的基本資料！其中 stk_list 是我們先前抓到的股票代碼喔！
stk_info_df = pd.DataFrame(index = stk_list.sort_values(), columns = info_columns)

# 創立一個紀錄失敗股票的 list
failed_list = []

# 開始迴圈抓資料囉！
for i in stk_info_df.index:
    try:
        # 打印出目前進度
        print('processing: ' + i)
        # 抓下來的資料暫存成 dictionary
        info_dict = yf.Ticker(i).info
        # 由於 yahoo finance 各檔股票所提供的欄位項目都不一致！所以這邊要針對每一檔股票分別取出欄位項目
        columns_included = list(info_dict.keys())
        # 因為在別檔公司裡有著 AAPL 裡所沒有的會計科目，因此要取兩家公司會計科目的交集
        intersect_columns = [x for x in info_columns if x in columns_included]
        # 有了該股欄位項目後，就可順利填入總表中相對應的位置
        stk_info_df.loc[i,intersect_columns] = list(pd.Series(info_dict)[intersect_columns].values)
        # 停一秒，再抓下一檔，避免對伺服器造成負擔而被鎖住
        time.sleep(1)
    except:
        failed_list.append(i)
        continue

# 查看一下資料內容，然後儲存下來吧！
stk_info_df.to_csv('')

ModuleNotFoundError: ignored

 GET 基本面資料

In [ ]:
import yfinance as yf
import time

# 先測試一檔試看看
stock = yf.Ticker('AAPL')

# 取得損益表，執行看看結果
stock.financials
import yfinance as yf
import time

# 先測試一檔試看看
stock = yf.Ticker('AAPL')

# 取得損益表，執行看看結果
stock.financials

# 取得資產負債表，執行看看結果
stock.balance_sheet

# 取得現金流量表，執行看看結果
stock.cashflow

# 開始迴圈抓資料囉！
for i in stk_list:
    try:
        # 打印出目前進度
        print('processing: ' + i)
        # 填入股票代碼後直接下載成 csv 格式
        stock = yf.Ticker(i)
        stock.financials.to_csv('profit_loss_account_'+i+'.csv')
        stock.balance_sheet.to_csv('balance_sheet_'+i+'.csv')
        stock.cashflow.to_csv('cash_flow_'+i+'.csv')
        # 停一秒，再抓下一檔，避免對伺服器造成負擔而被鎖住
        time.sleep(1)
    except :
        failed_list.append(i)
        continue

 GET 價量資料＋股利發放＋股票分割

In [3]:
import yfinance as yf
import time

# 先測試一檔試看看
stock = yf.Ticker('AAPL')

# 取得價量資料＋股利發放資料＋股票分割資料
stock.history(period = 'max')

# 創立一個紀錄失敗股票的 list
failed_list = []

# 開始迴圈抓資料囉！
for i in stk_list:
    try : 
        # 打印出目前進度
        print('processing: ' + i)
        # 填入股票代碼後直接下載成 csv 格式
        stock = yf.Ticker(i)
        stock.history(period = 'max').to_csv('price_'+i+'.csv')
        # 停一秒，再抓下一檔，避免對伺服器造成負擔而被鎖住
        time.sleep(1)
    except :
        failed_list.append(i)
        continue

ModuleNotFoundError: ignored